<a href="https://colab.research.google.com/github/Echo9k/Information-Retrival/blob/main/PA7_part4_WebCrawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web crawler
Information retrieval.



<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://rebrand.ly/PA7_part4_WebCrawler_colab"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://rebrand.ly/PA7_part4_WebCrawler_git"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://rebrand.ly/PA7_part4_WebCrawler_notebook"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>

  </td>
  <td>
    <a target="_blank" href="https://rebrand.ly/PA7_part4_WebCrawler_py"><img src="https://miro.medium.com/max/256/1*ztqS5rRI29GHxZa6uPF2UA.png" width="40" height="40" />Download .py</a>
  </td>
</table>

## Set up

In [76]:
#@markdown ### Import libriaries
import re
import tqdm
import time

import sys


from urllib.request import urlopen 
from urllib.error import URLError
from bs4 import BeautifulSoup 
import pandas as pd

In [81]:
#@title Class Crawler
# crawler
class Crawler():
    def __init__(self, domain, queue_limit=500):
        self.domain = domain
        self.queue=[domain]
        self.queue_limit = queue_limit
        self.info = {}  # Text just for the first few sites.
    
    def make_soup(self, URL)->BeautifulSoup:
        try:
            html = urlopen(URL)
            return BeautifulSoup(html, 'html.parser')
        except (TimeoutError, URLError) as err:
            print(f"error: {err} on {URL}")
            raise

    def get_urls(self, URL, include_external=False)-> (set, str):
        soup=self.make_soup(URL)
        regx_internal= re.compile('^(?!#)((?!\/\/).)*$')  # Incomplete links are internal
        regx_full_URL= re.compile('^(ht)(.)*$')

        internal=set([self.domain+link.attrs['href'] for link in soup.find_all('a', href=regx_internal) if 'href' in link.attrs])
        if include_external:
            full_URLs=set([link.attrs['href'] for link in soup.find_all('a', href=regx_full_URL) if 'href' in link.attrs])
            links = internal.union(full_URLs)
        else:
            links = internal
        
        return links, soup.text

    def queue_maker(self, queue_limit=500, quiet=False):
        while len(self.queue)<=queue_limit:
            URL = self.queue.pop(0)
            try: links, text = self.get_urls(URL)
            except (TimeoutError, URLError) as err: continue
            except: print(f"Unexpected error: {sys.exc_info()[0]}  while visiting {URL}"); continue
            self.info.update({URL: text})
            if links is not None:
                new_unique_URLs=set(self.queue).union(self.info)\
                                .symmetric_difference(links).intersection(links)
                self.queue.extend(new_unique_URLs)
                if not quiet: print(f"Queue size: {len(self.queue):<7} {len(new_unique_URLs):> 3} new unique URLs from  {URL: <7}")
        return self.queue, self.info

    def extractor(self):
        
        for i, URL in enumerate(self.queue,10):
            try:soup=self.make_soup(URL)
            except: print(f"Unexpected error: {sys.exc_info()[0]}  while visiting {URL}"); continue
            self.info.update({URL: soup.get_text(strip=True)})
            if i%100==0: print(f"{i}%")
            sys.stdout.write("-")
            sys.stdout.flush()

# Extract the info

In [ ]:
domain='http://www.hometownlife.com'  #@param{type:"string"}
limit_queue = 500 #@param {type:"slider", min:50, max:550, step:50}
crawler = Crawler(domain)
crawler.queue_maker(limit_queue)
print("extracting...")
crawler.extractor()

In [92]:
#@markdown Display data
%load_ext google.colab.data_table
data = pd.DataFrame(pd.Series(crawler.info))
data.reset_index(inplace=True)
data.columns=["url","text"]
# data.to_csv("data.csv")
data

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


# Statistics
Hello pears! I was planning into making a package out of the python Class from the [ notebook previous](https://colab.research.google.com/drive/14AV2jstGnxw67HaK7o6tZfAun0zVGx_s) but I could not replicate what I did on my computer here on colab.

Yet, this are the metrics I obtained.
|metric|value|
|:---|----|
Number of documents processed | 527
Total number of terms parsed from all documents | 1715640
Total number of unique terms found and added to the index | 35911
Total number of terms found that matched one of the stop words in your program’s stop words list | 263241